In [38]:
import keras
import tensorflow as tf
import os
from os.path import join
import json
import random
import itertools
import re
import numpy as np
import pylab
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model, load_model
from keras.layers.recurrent import GRU
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
import cv2

In [39]:
sess = tf.Session()
K.set_session(sess)

In [40]:
from collections import Counter
def get_counter(directory_path):
    dirname = os.path.basename(directory_path)
    ann_directory_path = join(directory_path, 'ann')
    letters = ''
    lens = []
    for filename in os.listdir(ann_directory_path):
        json_filepath = join(ann_directory_path, filename)
        description = json.load(open(json_filepath, 'r'))['description']
        lens.append(len(description))
        letters += description
    print('Max plate length in "%s":' % dirname, max(Counter(lens).keys()))
    return Counter(letters)

In [41]:
count_val = get_counter('./data/val/anpr_ocr/train/')
count_train = get_counter('./data/train/anpr_ocr/train/')
letters_train = set(count_train.keys())
letters_val = set(count_val.keys())
# print(len(letters_train), len(letters_val), len(letters_val | letters_train))
letters = sorted(list(letters_train))

Max plate length in "": 8
Max plate length in "": 8


In [42]:
def labels_to_text(labels):
    _map = map(lambda x: letters[int(x)], labels)
    return ''.join(list(_map))

def text_to_labels(text):
    _map = map(lambda x: letters.index(x), text)
    return list(_map)

def validate_string(s):
    for ch in s:
        if not ch in letters:
            return False
    return True

In [43]:
class GenerateDataset:
    
    def __init__(self, dirpath, width, height, batch_size, downsample_factor, max_text_len=8):
        
        self.height = height
        self.width = width
        self.batch_size = batch_size
        self.max_text_len = max_text_len
        self.downsample_factor = downsample_factor
        
        img_dirpath = join(dirpath, 'img')
        ann_dirpath = join(dirpath, 'ann')
        self.samples = []
        for filename in os.listdir(img_dirpath):
            name, ext = os.path.splitext(filename)
            if ext == '.png':
                img_filepath = join(img_dirpath, filename)
                json_filepath = join(ann_dirpath, name + '.json')
                description = json.load(open(json_filepath, 'r'))['description']
                if validate_string(description):
                    self.samples.append([img_filepath, description])
        
        self.n_samples = len(self.samples)
        self.indexes = list(range(self.n_samples))
        self.cur_index = 0
        
    def build_data(self):
        self.imgs = np.zeros((self.n_samples, self.height, self.width))
        self.texts = []
        for i, (img_filepath, text) in enumerate(self.samples):
            img = cv2.imread(img_filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (self.width, self.height))
            img = img.astype(np.float32)
            img /= 255
            # width and height are backwards from typical Keras convention
            # because width is the time dimension when it gets fed into the RNN
            self.imgs[i, :, :] = img
            self.texts.append(text)
        
    def get_output_size(self):
        return len(letters) + 1
    
    def next_sample(self):
        self.cur_index += 1
        if self.cur_index >= self.n_samples:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]
    
    def next_batch(self):
        while True:
            # width and height are backwards from typical Keras convention
            # because width is the time dimension when it gets fed into the RNN
            if K.image_data_format() == 'channels_first':
                X_data = np.ones([self.batch_size, 1, self.width, self.height])
            else:
                X_data = np.ones([self.batch_size, self.width, self.height, 1])
            Y_data = np.ones([self.batch_size, self.max_text_len])
            input_length = np.ones((self.batch_size, 1)) * (self.width // self.downsample_factor - 2)
            label_length = np.zeros((self.batch_size, 1))
            source_str = []
                                   
            for i in range(self.batch_size):
                img, text = self.next_sample()
                img = img.T
                if K.image_data_format() == 'channels_first':
                    img = np.expand_dims(img, 0)
                else:
                    img = np.expand_dims(img, -1)
                X_data[i] = img
                Y_data[i] = text_to_labels(text)
                source_str.append(text)
                label_length[i] = len(text)
                
            inputs = {
                'the_input': X_data,
                'the_labels': Y_data,
                'input_length': input_length,
                'label_length': label_length,
            }
            outputs = {'cost': np.zeros([self.batch_size])}
            yield (inputs, outputs)

In [48]:
def cost_function(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


def train(width, load=False):
    # Input Parameters
    height = 64

    # Network parameters
    conv_filters = 16
    kernel_size = (3, 3)
    pool_size = 2
    time_dense_size = 32
    rnn_size = 512

    if K.image_data_format() == 'channels_first':
        input_shape = (1, width, height)
    else:
        input_shape = (width, height, 1)
        
    batch_size = 32
    downsample_factor = pool_size ** 2
    train_data = GenerateDataset('./data/train/anpr_ocr/train/', width, height, batch_size, downsample_factor)
    train_data.build_data()
    val_data = GenerateDataset('./data/val/anpr_ocr/train/', width, height, batch_size, downsample_factor)
    val_data.build_data()

    act = 'relu'
    input_data = Input(name='the_input', shape=input_shape, dtype='float32')
    inner = Conv2D(conv_filters, kernel_size, padding='same',
                   activation=act, kernel_initializer='he_normal')(input_data)
    inner = MaxPooling2D(pool_size=(pool_size, pool_size))(inner)
    inner = Conv2D(conv_filters, kernel_size, padding='same',
                   activation=act, kernel_initializer='he_normal')(inner)
    inner = MaxPooling2D(pool_size=(pool_size, pool_size))(inner)

    conv_to_rnn_dims = (width // (pool_size ** 2), (height // (pool_size ** 2)) * conv_filters)
    inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)
    inner = Dense(time_dense_size, activation=act, name='dense1')(inner)
    gru_1 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal')(inner)
    gru_1b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal')(inner)
    merge_1 = add([gru_1, gru_1b])
    gru_2 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal')(merge_1)
    gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal')(merge_1)
    # transforms RNN output to character activations:
    inner = Dense(train_data.get_output_size(), kernel_initializer='he_normal')(concatenate([gru_2, gru_2b]))
    y_pred = Activation('softmax', name='softmax')(inner)
    Model(inputs=input_data, outputs=y_pred).summary()

    labels = Input(name='the_labels', shape=[train_data.max_text_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    # Keras doesn't currently support loss funcs with extra parameters
    # so CTC loss is implemented in a lambda layer
    loss_out = Lambda(cost_function, output_shape=(1,), name='cost')([y_pred, labels, input_length, label_length])

    # clipnorm seems to speeds up convergence
    sgd = keras.optimizers.sgd(lr=0.03, decay=1.5e-6, momentum=0.85, nesterov=True, clipnorm=5)

    if load:
        model = load_model('./ocr.h5', compile=False)
    else:
        model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
    model.compile(loss={'cost': lambda y_true, y_pred: y_pred}, optimizer=sgd)
    
    if not load:
        # captures output of softmax so we can decode the output during visualization
        test_func = K.function([input_data], [y_pred])

        model.fit_generator(generator=train_data.next_batch(), 
                            steps_per_epoch=train_data.n_samples,
                            epochs=1, 
                            validation_data=val_data.next_batch(), 
                            validation_steps=val_data.n_samples)

    return model

In [49]:
model = train(128, load=False)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
the_input (InputLayer)           (None, 128, 64, 1)    0                                            
____________________________________________________________________________________________________
conv2d_5 (Conv2D)                (None, 128, 64, 16)   160         the_input[0][0]                  
____________________________________________________________________________________________________
max1 (MaxPooling2D)              (None, 64, 32, 16)    0           conv2d_5[0][0]                   
____________________________________________________________________________________________________
conv2d_6 (Conv2D)                (None, 64, 32, 16)    2320        max1[0][0]                       
___________________________________________________________________________________________

ValueError: Unknown entry in loss dictionary: "cost". Only expected the following keys: ['ctc']